In [1]:
import threading
import itertools
import time
import sys
class Signal:
    go = True
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ''+ msg
        write(status)
        flush()
        write('\x08'* len(status))
        time.sleep(0.1)
        if not signal.go:
            break
    write(' '*len(status) + '\x08'* len(status))
def slow_function():
    time.sleep(3)
    return 42
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!',signal))
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result
def main():
    result = supervisor() 
    print('Answer:',result)
if __name__ == '__main__':
    main()

spinner object: <Thread(Thread-5, initial)>
|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin/thinkin-thinkin\thinkin|thinkin        Answer: 42


In [2]:
import asyncio
import itertools
import sys
async def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            await asyncio.sleep(.1)  # 1
        except asyncio.CancelledError:  # 2
            break
    write(' ' * len(status) + '\x08' * len(status))
async def slow_function():  # 3
    await asyncio.sleep(3)  # 4
    return 42
async def supervisor():  # 5
    spinner = asyncio.ensure_future(spin('thinking!'))  # 6
    print('spinner object:', spinner)  # 7
    result = await slow_function()  # 8
    spinner.cancel()  # 9
    return result
def main():
    loop = asyncio.get_event_loop()  # 10
    result = loop.run_until_complete(supervisor())  # 11
    loop.close()
    print('Answer:', result)
if __name__ == '__main__':
    main()

RuntimeError: This event loop is already running